# notebook in which all figures are created 

In [ ]:
include("../src/empirical_evd.jl")
using .empirical_evd
using Plots
using BPTT

function lorenz_regimes()
    tss = [create_series("lorenz", 20)]
    push!(tss, create_series("lorenz", 22))
    push!(tss, create_series("lorenz", 24))
    push!(tss, create_series("lorenz", 25))
    push!(tss, create_series("lorenz", 28))
    push!(tss, create_series("lorenz", 34))
    ps = (plot3d(tss[i][:, 1], tss[i][:, 2], tss[i][:, 3]) for i in 1:6)
    g = plot(ps..., layout=6, 
            title=["\n20" "ρ as bifurcation parameter\n22" "\n24" "\n25" "\n28" "\n34"], titlefont=font(11), 
            legend=nothing, 
            plot_title="3d lorenz regimes", plot_titlevspan=0.08)
    mkpath("../Figures/lorenz/") 
    savefig(g, "../Figures/lorenz/lorenz_regimes.png")
end

function bursting_neuron_regimes()
    tss = [create_series("bursting_neuron", 2)]
    push!(tss,  create_series("bursting_neuron", 3))
    push!(tss,  create_series("bursting_neuron", 5))
    push!(tss,  create_series("bursting_neuron", 9))
    push!(tss,  create_series("bursting_neuron", 10))
    push!(tss,  create_series("bursting_neuron", 10.2))
    ps = (plot3d(tss[i][:, 1], tss[i][:, 2], tss[i][:, 3]) for i in 1:6)
    g = plot(ps..., layout=6,
        title=["\n2" "\$g_{nmda}\$ as bifurcation parameter\n3" "\n5" "\n9" "\n10" "\n10.2"], titlefont=font(11),
        legend=nothing,
        plot_title="3d bursting neuron regimes", plot_titlevspan=0.08,
        size=(500, 500)
    )
    mkpath("../Figures/bursting_neuron/")
    savefig(g, "../Figures/bursting_neuron/bursting_neuron_regimes.png")
end

In [7]:
include("../src/empirical_evd.jl")
using .empirical_evd
using Plots
using BPTT
using LinearAlgebra

function lorenz_evd()
    function remove!(a, item)
        deleteat!(a, findall(x -> x == item, a))
    end

    experiments = readdir("../Results")
    remove!(experiments, "default")
    deleteat!(experiments, findall(x -> occursin("bursting", x), experiments))

    # load all the experiments
    for exp in experiments
        model_path = load_result_path(exp, 1000, 1)
        m = BPTT.load_model(model_path)

        # compose single matrix AW
        AW = diagm(m.A) + m.W

        # get evd of AW
        EVD = eigen(AW)
        λs = EVD.values
        Vs = EVD.vectors
        try
            push!(exp_λs, λs)
            push!(exp_Vs, Vs)
        catch e
            global exp_λs = [λs]
            global exp_Vs = [Vs]
        end
    end

    # for making subplots for each experiment
    function make_subplots(exp, plots, type)
        if exp == "lorenz"
            μ = "ρ"
            title = ["\n20" "ρ as bifurcation parameter\n22" "\n24" "\n25" "\n28" "\n34"]
        else
            μ = "\$g_{nmda}\$"
            title = ["\n2" "$μ as bifurcation parameter\n3" "\n5" "\n9" "\n10" "\n10.2"]
        end
        p = plot(plots..., layout=grid(1, 6),
            title=title, titlefont=font(11),
            legend=nothing,
            plot_title="$type of evd with $exp plrnn", plot_titlevspan=0.08
        )
        display(p)
    end

    # convert to array
    arr_exp_l = reduce(hcat, exp_λs)

    # Plots of single experiments

    # sort the absolute values for comparison
    abs_λ = [sort(abs.(lambdas)) for lambdas in exp_λs]

    # histogram of all abs values
    hists_abs_val = (histogram(abs_val, bins=8) for abs_val in abs_λ)

    # im and real part in one figure
    for lambdas in exp_λs
        imre = scatter(imag(lambdas), label="imag")
        scatter!(imre, real(lambdas), label="real")
        try
            push!(im_res, imre)
        catch
            global im_res = [imre]
        end
    end

    # abs values together
    for abs_val in abs_λ
        id = 1
        try
            scatter!(abss, abs_val, 1:15, label="run $id")
        catch
            global abss = scatter(abs_val, 1:15, label="run $id")
        end
        id+=1
    end
    savefig(abss, "lorenz/change_in_abs_vals.png")


    make_subplots("lorenz", hists_abs_val, "hist of abs val")
    make_subplots("lorenz", im_res, "im and real")

    # look at change of the parameters
    for id in 2:3#6 # for all experiments
        try
            push!(change_abs, abs_λ[id] ./ abs_λ[id-1])
        catch e
            global change_abs = [abs_λ[id] ./ abs_λ[id-1]]
        end
    end
    arr_change_abs = reduce(hcat, change_abs)

    # hist of overall change in all eigenvalues
    hist_abs = histogram(collect(Iterators.flatten(change_abs) .- 1),
        bins=15,
        title="change of all eigenvalues",
        legend=nothing)
    savefig(hist_abs, "lorenz/overall_cahnge_abs_val.png")

    # hist of change of the values
    hist_lambda_abs = scatter(sum(arr_change_abs, dims=2), title="sum of change of eigenvalues", legend=nothing)
    vline!(hist_lambda_abs, [12, 13], c=:blue, linewidth=1)
    savefig(hist_lambda_abs, "lorenz/change_of_ev.png")

    # plot the change of some of the imoprtant eigenvalues
    val, idx = findmax(sum(arr_change_abs, dims=2))
    arr_abs_λ = reduce(hcat, abs_λ)

    #plots have to be adjusted manually to see the ones crossing 1
    a = plot(arr_abs_λ[12, :])
    b = plot(arr_abs_λ[13, :])
    c = plot(arr_abs_λ[idx[2], :])
    d = plot(a, b, c, layout=grid(1, 3), size=(800, 300))
    savefig(d, "lorenz/ev_t_cross1.png")

    # heatmap for all eigenvalues
    heatmap(abs.(arr_abs_λ), c=:thermal)
    savefig(heat, "lorenz/heatmap.png")
end

lorenz_evd (generic function with 1 method)

In [8]:
lorenz_evd()
